In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import  ChatPromptTemplate, MessagesPlaceholder


llm  = ChatOpenAI(model_name="gpt-4o-mini",
                temperature=0.1,
                streaming=True,
                )

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    #memory_key="chat_history", #기본 키는 history다 
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI taking to a human."),
        MessagesPlaceholder(variable_name="history"), #message형식일 때 히스토리 제공방법
        ("human", "{question}"),
    ]
)
def load_memory(_):
    return memory.load_memory_variables({})["history"]
#RunnablePassthrough : 프롬프트가 완성되기 전에 함수를 실행시켜줌
#그리고 assign 메소드를 통해 어느 값이든 할당할 수 있음
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question,
    })
    memory.save_context({"input": question},{"output": result.content}) 
    print(result)




In [14]:
invoke_chain("My name is jonas")


content='Nice to meet you, Jonas! How can I assist you today?' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0'} id='run-5556a100-a4d5-494e-9efa-00aaa16821af-0'


In [15]:
invoke_chain("I live in Seoul")


content="That's great! Seoul is a vibrant city with a rich history and a lot to offer. Do you have any favorite places or activities in Seoul?" additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0'} id='run-2f0e540b-f3b6-472a-8117-a8272c4e2bf9-0'
